## Day 46 Lecture 2 Assignment

In this assignment, we will calculate a similarity matrix using responses to a survey about student life at a university. We will perform clustering on this dataset later on.

In [116]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [117]:
def get_df(url):
  df = pd.read_csv(url)
  return df

In [118]:
# answer goes here
sl = get_df('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
sl.head()



,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,5


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [119]:
# answer goes here
stress_id = 'Q5'
stress_questions = [q for q in sl.keys() if stress_id in q] 
sl = sl[stress_questions]
sl.head()


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0


We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

The pdist function is a simple way to accomplish this goal, and the squareform function will transform the output of the pdist function into a tabular format that can be convert to a dataframe more easily.

In [120]:
# answer goes here
dm = pdist(sl, metric='hamming')
dm = pd.DataFrame(squareform(dm))
dm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,0.2,0.2,0.4,0.5,0.3,0.2,0.2,0.3,0.3,0.4,0.2,0.3,0.2,0.1,0.2,0.3,0.2,0.1,0.4,0.0,0.2,0.2,0.4,0.2,0.4,0.0,0.2,0.2,0.4,0.1,...,0.3,0.1,0.1,0.0,0.1,0.3,0.0,0.1,0.2,0.1,0.3,0.1,0.2,0.3,0.2,0.1,0.1,0.0,0.1,0.0,0.2,0.2,0.1,0.3,0.1,0.2,0.3,0.0,0.3,0.1,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,0.7,0.5,0.5,0.4,0.6,0.7,0.7,0.6,0.6,0.7,0.7,0.6,0.7,0.6,0.5,0.6,0.7,0.8,0.5,0.7,0.7,0.7,0.7,0.5,0.5,0.7,0.7,0.7,0.7,0.6,...,0.6,0.6,0.8,0.7,0.6,0.4,0.7,0.6,0.7,0.6,0.4,0.6,0.5,0.6,0.5,0.8,0.6,0.7,0.6,0.7,0.5,0.5,0.6,0.6,0.6,0.7,0.6,0.7,0.4,0.6,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,0.4,0.4,0.4,0.5,0.5,0.6,0.4,0.5,0.3,0.4,0.2,0.3,0.4,0.3,0.4,0.3,0.2,0.3,0.4,0.4,0.4,0.2,0.4,0.4,0.4,0.4,0.2,0.4,0.4,0.3,...,0.3,0.3,0.3,0.4,0.3,0.3,0.4,0.5,0.2,0.3,0.3,0.3,0.2,0.3,0.4,0.5,0.5,0.4,0.3,0.4,0.4,0.4,0.3,0.5,0.3,0.2,0.3,0.4,0.3,0.5,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,0.2,0.2,0.3,0.6,0.3,0.3,0.2,0.5,0.4,0.4,0.3,0.5,0.3,0.4,0.4,0.5,0.1,0.6,0.2,0.3,0.3,0.2,0.3,0.2,0.5,0.0,0.3,0.2,0.1,0.3,0.3,0.3,0.5,0.3,0.5,0.3,0.1,0.3,0.3,0.2,...,0.2,0.4,0.2,0.3,0.2,0.6,0.3,0.4,0.1,0.2,0.6,0.2,0.3,0.4,0.3,0.4,0.2,0.3,0.2,0.3,0.5,0.1,0.2,0.4,0.2,0.1,0.2,0.3,0.4,0.4,0.1,0.1,0.4,0.3,0.5,0.0,0.2,0.2,0.4,0.2
2954,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2955,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2956,0.2,0.2,0.1,0.6,0.5,0.3,0.2,0.3,0.6,0.4,0.3,0.3,0.5,0.4,0.4,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.2,0.3,0.4,0.3,0.2,0.5,0.1,0.1,0.3,0.5,0.3,0.5,0.1,0.3,0.3,0.5,0.2,...,0.4,0.2,0.2,0.1,0.2,0.4,0.1,0.0,0.3,0.2,0.4,0.2,0.3,0.4,0.1,0.2,0.2,0.1,0.2,0.1,0.3,0.3,0.2,0.4,0.2,0.3,0.4,0.1,0.2,0.0,0.3,0.5,0.4,0.5,0.5,0.4,0.2,0.2,0.0,0.2


Print the answers to the following three questions:

1. How many pairs of students are there in our dataset? (Remember to ignore a student pairing with themselves, and avoid double-counting - '3-5' is the same as '5-3'.) 
2. Of these pairs, how many gave identical answers? 
3. How many pairs had no answers in common?

Tip: it may help to first convert the NxN similarity matrix into a table with three columns: response_id_1, response_id_2, and the similarity between them. The new table should have NxN rows and 3 columns.



```
# Table Pseudo Code
loop through the distance matrix index:
  for every index (as j in i in distance matrix)
    if the j != distance matrix name
      if the pair is not in the similarity table (st)
        add both ids and the distance to the table.       
```

The above pseudocode looked great in theory, but it didn't work out for me. Switched to melt, see below. 

Pulled from slack:

Mike Swirsky  1:01 PM
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html
:+1:
1






1:04
Further hints for getting the similarity matrix into shape: create a new column called 'id' that matches the index, then use that column as the id_vars in melt, and all other columns as the value_vars
:+1::skin-tone-6:
1

1:05
remember to remove all cases where the student is the same, and also to remove duplicates (student 1 with student 2, student 2 with student 1)
1:06
there is a slick way to do both of those at once with loc


In [121]:
dm['id'] = dm.index
st = pd.melt(dm, id_vars=['id'])
st.head()

,id,variable,value
0,0,0,0.0
1,1,0,0.0
2,2,0,0.1
3,3,0,0.6
4,4,0,0.3


In [122]:
st = st.drop_duplicates(subset=['id', 'variable'], keep='last')

In [123]:
st = st[st['id'] != st['variable']]
st.head()

,id,variable,value
1,1,0,0.0
2,2,0,0.1
3,3,0,0.6
4,4,0,0.3
5,5,0,0.1


In [124]:
#Pairs in the dataset (or, moreover); how many pairs of students.
st.count()

id          8746806
variable    8746806
value       8746806
dtype: int64

In [125]:
#how many gave identical answers?
st[st['value'] == 0].count()

id          273464
variable    273464
value       273464
dtype: int64

In [126]:
#How many had nothing in common?
st[st['value'] == 1].count()

id          582
variable    582
value       582
dtype: int64

For each student, calculate their average similarity with all other students. Then print the following:

1. On average, which student (i.e. which response_id) had the most similar responses to the survey population as a whole? What was the average similarity for this student, and what were their answers?
2. Repeat part 1, but for the student with the least similar responses to the survey population as a whole.
3. What do we notice about these students? What broader inferences can we make about the student population as a whole?

In [127]:
# answer goes here
st.groupby(by='id').mean().sort_values(by='value', ascending=False)

,value
id,
2899,0.624146
2416,0.601353
923,0.593777
1338,0.593777
1605,0.593777
...,...
1529,0.224822
1520,0.224822
2675,0.224822


In [130]:
sl[sl.index == 2899]

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
2899,1,1,1,1,1,0,1,1,1,1


In [131]:
# answer goes here
st.groupby(by='id').mean().sort_values(by='value', ascending=True)

,value
id,
0,0.224822
440,0.224822
1865,0.224822
1869,0.224822
433,0.224822
...,...
866,0.593777
878,0.593777
1796,0.593777


In [132]:
sl[sl.index == 0]

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0


Answer to question number three!

Oh. You're asking me to make an assumption. It's not an assumption really. I'm looking at a situation where the bulk of the answers are identical. The one person with the least amount in common with everyone has at least something in common with the one who has the most in common with everyone and that would be that their stressed out about academic issues. 

I would infer that there was a high number of people that were stressed out about or by school. They were under a lot of academic pressure. 

In [134]:
sl.describe()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
count,2958.000000,2958.000000,2958.000000,2958.000000,2958.000000,2958.000000,2958.000000,2958.000000,2958.000000,2958.000000
mean,0.306626,0.922245,0.386072,0.147397,0.269777,0.165652,0.151116,0.455037,0.274172,0.013861
std,0.461171,0.267831,0.486930,0.354561,0.443919,0.371831,0.358222,0.498058,0.446171,0.116933
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
